Universidad del Valle de Guatemala

## Proyecto: Data Science

Andrea Amaya 19357 | Brandon Hernández	 19376 |		Martin Amado	19020 |   Juan Pablo Pineda 19087

In [ ]:
# run once
import glob, os
os.chdir('./cvs_data')

In [ ]:
import pandas as pd 
import re
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None 

In [ ]:
data_frames = []
for file in glob.glob('*.csv'):
  print(file.split('.').pop(0))
  data_frames.append(pd.read_csv(file, encoding='utf-8'))

In [ ]:
df1 = data_frames[0]
df1 = df1.apply(lambda x: x.astype(str).str.lower()) # Todo a minusculas
df1.shape[0]

#### Limpiando los numeros de telefono

In [ ]:
def cleanNumbers(number):
    result =  re.sub("[^0-9]", "", number) # Dejar solo los numeros

    # Cada numero de celular es de 8 digitos
    n = 8 
    possibleNumbers = [result[i:i+n] for i in range(0, len(result), n)]

    # Revisar que la validacion sea correcta
    for number in possibleNumbers:
        if len(str(number)) != 8:
            print('Opps! Fallo la validacion de numeros', number)
            return False
    
    return possibleNumbers

In [ ]:

for index, row in df1.iterrows():
    resultPhone = cleanNumbers(row["TELEFONO"])
    if(resultPhone != False):
        row["TELEFONO"] = resultPhone

df1 = df1.explode('TELEFONO')

In [ ]:
df1.shape[0]